# Learning Python (Day 53)

# Python Flask: User Profiles

In this, the focus is on creating user profiles within a Flask application. This involves extending the `User` model to include additional profile information, setting up profile pages, and handling profile updates. Here’s a detailed breakdown:

## Extending the User Model

The `User` model is extended to include more fields related to the user's profile. This could include fields like name, location, about_me, and member_since.

```python
class User(db.Model):
    __tablename__ = 'users'
    id = db.Column(db.Integer, primary_key=True)
    username = db.Column(db.String(64), unique=True, index=True)
    email = db.Column(db.String(64), unique=True, index=True)
    password_hash = db.Column(db.String(128))
    name = db.Column(db.String(64))
    location = db.Column(db.String(64))
    about_me = db.Column(db.Text())
    member_since = db.Column(db.DateTime(), default=datetime.utcnow)
    last_seen = db.Column(db.DateTime(), default=datetime.utcnow)

    def __repr__(self):
        return '<User {}>'.format(self.username)
```

## User Loader

To load a user from the database by ID, the `user_loader` callback is used. This allows Flask-Login to retrieve the user details.

```python
@login.user_loader
def load_user(user_id):
    return User.query.get(int(user_id))
```

## Profile Page

The profile page displays the user’s information. A route is created to handle requests for a user’s profile.

```python
@main.route('/user/<username>')
def user(username):
    user = User.query.filter_by(username=username).first_or_404()
    return render_template('user.html', user=user)
```

## Last Seen

To keep track of the last time a user was seen, a timestamp is updated every time a user makes a request.

```python
@app.before_request
def before_request():
    if current_user.is_authenticated:
        current_user.ping()
```

In the `User` model, the `ping` method is defined to update the last_seen field.

```python
class User(db.Model):
    # ... other fields and methods ...
    
    def ping(self):
        self.last_seen = datetime.utcnow()
        db.session.add(self)
```

## Profile Edit Form

A form is created to allow users to edit their profiles. Flask-WTF is used to handle form creation and validation.

```python
class EditProfileForm(FlaskForm):
    name = StringField('Real name', validators=[Length(0, 64)])
    location = StringField('Location', validators=[Length(0, 64)])
    about_me = TextAreaField('About me')
    submit = SubmitField('Submit')
```

## Edit Profile Route

A route and corresponding view function are created to handle profile edits.

```python
@main.route('/edit-profile', methods=['GET', 'POST'])
@login_required
def edit_profile():
    form = EditProfileForm()
    if form.validate_on_submit():
        current_user.name = form.name.data
        current_user.location = form.location.data
        current_user.about_me = form.about_me.data
        db.session.add(current_user)
        flash('Your profile has been updated.')
        return redirect(url_for('.user', username=current_user.username))
    form.name.data = current_user.name
    form.location.data = current_user.location
    form.about_me.data = current_user.about_me
    return render_template('edit_profile.html', form=form)
```

## Templates

Templates for displaying and editing the user profile are created. The `user.html` template shows the user’s profile, while the `edit_profile.html` template provides the form for editing the profile.

```html
<!-- user.html -->
{% extends "base.html" %}
{% block content %}
    <h1>{{ user.username }}</h1>
    <p>{{ user.name }}</p>
    <p>{{ user.location }}</p>
    <p>{{ user.about_me }}</p>
    <p>Member since: {{ user.member_since }}</p>
    <p>Last seen: {{ user.last_seen }}</p>
{% endblock %}

<!-- edit_profile.html -->
{% extends "base.html" %}
{% block content %}
    <h1>Edit Profile</h1>
    <form method="post">
        {{ form.hidden_tag() }}
        {{ form.name.label }} {{ form.name(size=32) }}<br>
        {{ form.location.label }} {{ form.location(size=32) }}<br>
        {{ form.about_me.label }} {{ form.about_me(rows=4) }}<br>
        {{ form.submit() }}
    </form>
{% endblock %}
```

## Summary

This covers how to enhance user profiles in a Flask application by extending the user model, creating profile pages, and enabling profile edits. By following these steps, users can have a richer and more personalized experience within the application. 

The focus on extending the user model, updating profile data, and ensuring the information is displayed and editable through user-friendly forms and templates is crucial for developing interactive and engaging web applications.